# Dojo

Dojo is a molecular mechanics simulation engine. It uses OpenMM to run molecular dynamics simulations with many additional features added by the Rush team as well.

## Module Specification

The module takes an `dict` of module-specific options, a `[TRObject]` representing the molecules to include in the simulation, and an `[Object<XML>]` that corresponds to additional force field parameters used for simulating smols accurately. These XML files can be generated using the ffgen module.

### Inputs

* options: `dict`
* trcs: `[TRObject]`
* force_fields: `[Object<XML>]` - one for each smol appearing in the input TRCObjects

### Outputs

* final_frame_trc: `[TRObject]` - representing the state of each input on the simulation's final frame
* frames: `Object<[dict]>` - stores the requested data for the output frames of the simulation as per the configuration

## Options

Dojo has a lot of options!

The following documents the structure of the config dict along with what each field represents and possible default values.

This markdown file is broken down into sections, one for each non-trivial type defined.
Within a section for a given type, there will be a schematic description of the structure of the type, followed by a description of each of the fields.
In the description of the fields, there will be given its name, type, default value (if any) and a short description.
The type can either be a basic type (`int`, `float`, `bool`, `list<T>`) or a non-trivial type, in which case the name of the type will be written in CamelCase and will have a corresponding section in this file.
If a default for a field is given, no value need be provided for this field, and if none is provided it will assume the value of the default.

Any parameters marked with an asterisk (*) are either unused, unimplemented, broken, or not relevant when called as a module.

### Parameters

The top level structure that is passed to Dojo.

```
Parameters
 ├ num_gpus
 ├ single_frame_serialise
 ├ nonbonded_method
 ├ nonbonded_cutoff_nm
 ├ constraints
 ├ remove_cm_motion
 ├ generate_ff_indices
 ├ force_fields
 ├ qm_region
 │ ├ engine
 │ └ regions
 ├ qm_cutoff_distance_ang
 ├ simulation
 │ ├ step_size_ps
 │ ├ periodic_box_vectors_nm
 │ │ ├ v1
 │ │ ├ v2
 │ │ └ v3
 │ └ stages
 ├ implicit_solvent
 ├ solvation
 ├ metadynamics
 ├ bias_potentials
 ├ bias_schedule
 │ ├ active_potentials
 │ └ triggers
 ├ termination_condition
 │ ├ tail_steps
 │ └ bonds
 ├ frames
 │ ├ start
 │ ├ stop
 │ └ interval
 ├ trajectory
 │ ├ interval
 │ ├ minimisation
 │ ├ forces
 │ ├ info
 │ ├ windowed
 │ ├ window_size
 │ └ residues
 ├ save_final_trcs
 ├ save_final_velocities
 └ qdxf_export
```

| Name                    | Type                  | Default                             | Description                                                                                                                              |
|-------------------------|-----------------------|-------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------|
| num_gpus*               | int                   | 1                                   | Number of GPUs to use for the calculation. When called as a module, this is ignored; please use the Resources struct instead.            |
| single_frame_serialise* | bool                  | false                               | Save the positions and forces of the initial system in a special format. Not currently wired up to the module.                           |
| nonbonded_method        | string                | "NoCutoff"                          | Which cutoff method to use for nonbonded forces. Options are "NoCutoff", "CutoffNonPeriodic", "CutoffPeriodic", "Ewald", "PME", "LJPME". |
| nonbonded_cutoff_nm     | float                 | 1.0                                 | Cutoff for the nonbonded forces, in nm.                                                                                                  |
| constraints             | string                | "None"                              | Which constraints to use. Options are "None", "HBonds", "AllBonds", "HAngles".                                                           |
| remove_cm_motion        | bool                  | true                                | Whether or not to remove the centre of mass motion.                                                                                      |
| generate_ff_indices*    | list\<int\>           | []                                  | Which input files to generate FF parameters for. Currently not supported.                                                                |
| force_fields            | list\<string\>        | [ "Ff14sb", "Tip3pStandard" ]       | Which force fields to use for the simulation. Options are "Ff14sb", "Tip3pStandard" and "Custom".                                        |
| qm_region               | QMRegion              | { "engine": "None", "regions": [] } | Definition of the QM region.                                                                                                             |
| qm_cutoff_distance_ang  | float                 | 0.0                                 | All residues within this distance of the QM region will also be added to the QM region. Units of angstroms.                              |
| simulation              | Simulation            |                                     | Definition of the simulation.                                                                                                            |
| implicit_solvent*       | bool                  | false                               | Whether or not to use an implicit solvent. Currently not working.                                                                        |
| solvation               | SolvationParameters   | null                                | Definition of the solvation for the system.                                                                                              |
| metadynamics            | list\<Metadynamics\>  | []                                  | Definition of any metadynamics used for the simulation.                                                                                  |
| bias_potentials         | list\<BiasPotential\> | []                                  | List of bias potentials to use in the simulation.                                                                                        |
| bias_schedule           | BiasSchedule          | null                                | Definition of the schedule for the bias potentials.                                                                                      |
| termination_condition   | TerminationCondition  | null                                | Definition of the termination condition for the simulation.                                                                              |
| frames*                 | Frames                | null                                | Definition of output frames. Currently unused.                                                                                           |
| trajectory              | Trajectory            | null                                | Definition of the output trajectory. This trajectory is the main result obtained from running the module.                                |
| save_final_trcs*        | bool                  | false                               | Whether or not to save the structures that are the final geometry of the simulation. Currently not output by the module, even if set.    |
| save_final_velocities*  | bool                  | false                               | Whether or not to save the velocities in the outputs generated if `save_final_trcs` is enabled.                                          |
| qdxf_export*            | QDXFExport            | null                                | Whether or not to save outputs that can be used as inputs to PBSA calculations. Currently not output by the module, even if set.         |

### QMRegion

A description of how the QM region is evaluated and which parts of the system are included in the QM region for QM/MM simulations.

```
QMRegion
 ├ engine
 └ regions
```

| Name    | Type                  | Default | Description                                                                                                                  |
|---------|-----------------------|---------|------------------------------------------------------------------------------------------------------------------------------|
| engine  | string                |         | Which engine to use for the QM calculations. Options are "None", "QChem", "Xtb", "Exess". Currently only "Xtb" is supported. |
| regions | list\<QMRegionEntry\> |         | Which parts of the system are included in the QM region.                                                                     |

### QMRegionEntry

A description of which parts of the system constitute the QM region.

```
QMRegionEntry
 ├ file_index
 └ residues
```

| Name       | Type        | Default | Description                                                        |
|------------|-------------|---------|--------------------------------------------------------------------|
| file_index | int         |         | Index of the file supplied to Dojo, starting from 0.               |
| residues   | list\<int\> |         | List of residue indices for the QM residues. Indices start from 0. |

### Simulation

The parameters for the simulation.
At the top level ther are parameters that apply to the whole simulation, and then the sequence of simulation stages that are to occur

```
Simulation
 ├ step_size_ps
 ├ periodic_box_vectors_nm
 │ ├ v1
 │ ├ v2
 │ └ v3
 └ stages
```

| Name                    | Type                    | Default | Description                                      |
|-------------------------|-------------------------|---------|--------------------------------------------------|
| step_size_ps            | float                   |         | The step size to use for dynamics, in ps.        |
| periodic_box_vectors_nm | BoxVectors              | null    | Vectors to use for periodic boundary conditions. |
| stages                  | list\<SimulationStage\> |         | Sequence of simulation stages.                   |

### BoxVectors

Collection of three vectors defining a box.

```
BoxVectors
 ├ v1
 ├ v2
 └ v3
```

| Name | Type          | Default | Description                                                        |
|------|---------------|---------|--------------------------------------------------------------------|
| v1   | list\<float\> |         | First vector. Expected to have exactly 3 coordinates in the list.  |
| v2   | list\<float\> |         | Second vector. Expected to have exactly 3 coordinates in the list. |
| v3   | list\<float\> |         | Third vector. Expected to have exactly 3 coordinates in the list.  |

### Simulation Stage

Description of an individual stage of the simulation.

This type represents a discrimated union, and so only one of the fields may be present.

```
SimulationStage
 ├ Minimisation
 ├ Nvt
 └ Npt
```

| Name         | Type                   | Default | Description                           |
|--------------|------------------------|---------|---------------------------------------|
| Minimisation | MinimisationParameters |         | Parameters for a minimisation stage.  |
| Nvt          | NvtParameters          |         | Parameters for an NVT ensemble stage. |
| Npt          | NptParameters          |         | Parameters for an NPT ensemble stage. |

### MinimisationParameters

System minimisation stage.

```
MinimisationParameters
 ├ qm_minimisation_technique
 ├ max_iterations
 ├ err_tol_kj_per_mol_per_nm
 └ use_bias
```

| Name                      | Type   | Default | Description                                                                                                                                                                                                                                                                                                                                        |
|---------------------------|--------|---------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| qm_minimisation_technique | string | "None"  | Minimisation technique to use when there is a QM region present. Options are "None", "Dynamics" and "Bfgs". "Bfgs" is the BFGS gradient descent algorithm. "Dynamics" is an approximate technique that propagates dynamics, scaling the velocities down at each step. If "None" is selected and the system has a QM region, there will be an error |
| max_iterations            | int    | 0       | Maximum number of iterations to run for minimisation. A value of 0 means that it will iterate until the err tolerance is met. If there is a QM region behaviour is uncertain.                                                                                                                                                                      |
| err_tol_kj_per_mol_per_nm | float  | 10.0    | Error tolerance for the RMSD of forces in the system that will trigger the end of minimisation. Units are kJ/mol/nm. May not actually be wired up to the module properly.                                                                                                                                                                          |
| use_bias                  | bool   | false   | Whether or not to include bias potentials during the minimisation.                                                                                                                                                                                                                                                                                 |

### NvtParameters

NVT ensemble stage.

```
NvtParameters
 ├ temperature_kelvin
 ├ steps
 ├ starting_temperature_kelvin
 └ temperature_step
```

| Name                        | Type  | Default                                                    | Description                                                                                                                                      |
|-----------------------------|-------|------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| temperature_kelvin          | float |                                                            | Temperature at which to run the dynamics, in Kelvin.                                                                                             |
| steps                       | int   |                                                            | Number of steps to run the stage for.                                                                                                            |
| starting_temperature_kelvin | float | `temperature_kelvin`                                       | Starting temperature for a gradual temperature increase. If set equal to `temperature_kelvin`, temperature will stay constant during this stage. |
| temperature_step            | float | `0.1 * (temperature_kelvin - starting_temperature_kelvin)` | How much to change the thermostat at each step of the gradual increase.                                                                          |

### NptParameters

NPT ensemble stage.


```
NptParameters
 ├ temperature_kelvin
 ├ pressure_atm
 └ steps
```


| Name               | Type  | Default | Description                                            |
|--------------------|-------|---------|--------------------------------------------------------|
| temperature_kelvin | float |         | Temperature at which to run the dynamics, in Kelvin.   |
| pressure_atm       | float |         | Pressure at which to run the dynamics, in atmospheres. |
| steps              | int   |         | Number of steps to run the stage for.                  |

### SolvationParameters

Description for solvent added to the system.

This type represents a discrimated union, and so only one of the fields may be present.

```
BoxSizeNm
 ├ BoxSizeNm
 ├ BoxVectorsNm
 ├ PaddingNm
 └ NumAdded
```

| Name         | Type          | Default | Description                                                               |
|--------------|---------------|---------|---------------------------------------------------------------------------|
| BoxSizeNm    | list\<float\> |         | Dimensions of the final solvent box, in nm. Not current supported.        |
| BoxVectorsNm | BoxVectors    |         | List of vectors describing the solvent box, in nm. Not current supported. |
| PaddingNm    | float         |         | Solvent padding to add around the system, in nm.                          |
| NumAdded     | int           |         | Number of solvent atoms to add. Not current supported.                    |

### Metadynamics

Description of metadynamics that will be used during the simulation.

```
Metadynamics
 ├ bias_period
 ├ exp_sigma
 ├ height
 ├ csv_output
 └ cvs
```

| Name        | Type                    | Default | Description                                                                                                                                       |
|-------------|-------------------------|---------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| bias_period | int                     |         | Number of steps between adding exponentials to the bias potential.                                                                                |
| exp_sigma   | float                   |         | Sigma for the exponential functions that are added to the bias, where the exponential functions have the form form `h * exp(-0.5 * x^2 / sigma)`. |
| height      | MTDHeight               |         | Height of the exponential functions added to the bias potential.                                                                                  |
| csv_output  | string                  | null    | Filename for the CSV formatted representation of the final bias potential at the end of the simulation. Not currently wired up to the module.     |
| cvs         | list\<MTDCVParameters\> |         | List of collective variables used to define the metadynamics.                                                                                     |

### MTDHeight

Definition of the height of the exponential functions that are added to the bias potential during metadynamics.

This type represents a discrimated union, and so only one of the fields may be present.

```
MTDHeight
 ├ Constant
 └ WellTempered
```

| Name         | Type         | Default | Description                                    |
|--------------|--------------|---------|------------------------------------------------|
| Constant     | Constant     |         | Parameters for a constant height exponential.  |
| WellTempered | WellTempered |         | Parameters for an adaptive height exponential. |

### Constant

Exponential height that is a fixed constant.

```
Constant
 └ height
```

| Name   | Type  | Default | Description                             |
|--------|-------|---------|-----------------------------------------|
| height | float |         | Height of each exponential to be added. |

### WellTempered

Adaptive exponential height based on the well tempered metadynamics protocol.

```
MTDHeight
 ├ delta_t
 └ initial_deposition_rate
```

| Name                    | Type  | Default | Description                                       |
|-------------------------|-------|---------|---------------------------------------------------|
| delta_t                 | float |         | Delta T parameter from the paper.                 |
| initial_deposition_rate | float |         | Initial deposition rate parameter from the paper. |

### MTDCVParameters

Parameters for a given collective variable.

```
MTDCVParameters
 ├ num_samples
 ├ min
 ├ max
 └ variable
```

| Name        | Type        | Default | Description                                                                      |
|-------------|-------------|---------|----------------------------------------------------------------------------------|
| num_samples | int         |         | Number of sample points in the discretised representation of the bias potential. |
| min         | float       |         | Minimum of the range of the CV.                                                  |
| max         | float       |         | Maximum of the range of the CV.                                                  |
| variable    | CV          |         | Description of the CV.                                                           |

### CV

Definition of a collective variable (CV).

This type represents a discrimated union, and so only one of the fields may be present.

```
CV
 ├ Bond
 ├ Dihedral
 ├ Centroid
 └ RMSD
```

| Name     | Type     | Default | Description           |
|----------|----------|---------|-----------------------|
| Bond     | Bond     |         | Bond length CV.       |
| Dihedral | Dihedral |         | Dihedral angle CV.    |
| Centroid | Centroid |         | Centroid distance CV. |
| RMSD     | RMSD     |         | RMSD CV.              |

### Bond

Bond between two atoms.

```
Bond
 ├ index1
 └ index2
```

| Name   | Type | Default | Description                                      |
|--------|------|---------|--------------------------------------------------|
| index1 | int  |         | Atom index of the first atom defining the bond.  |
| index2 | int  |         | Atom index of the second atom defining the bond. |

### Dihedral

Dihedral angle CV.

```
Dihedral
 ├ index1
 ├ index2
 ├ index3
 └ index4
```

| Name   | Type | Default | Description                                          |
|--------|------|---------|------------------------------------------------------|
| index1 | int  |         | Atom index of the first atom defining the dihedral.  |
| index2 | int  |         | Atom index of the second atom defining the dihedral. |
| index3 | int  |         | Atom index of the third atom defining the dihedral.  |
| index4 | int  |         | Atom index of the fourth atom defining the dihedral. |

### Centroid

Centroid distance CV.

This type represents a discrimated union, and so only one of the fields may be present.

```
Centroid
 ├ Indices
 ├ Residues
 └ Files
```

| Name     | Type     | Default | Description                              |
|----------|----------|---------|------------------------------------------|
| Indices  | Indices  |         | Centroid distance based on atom indices. |
| Residues | Residues |         | Centroid distance based on residues.     |
| Files    | Files    |         | Centroid distance based on files.        |

### Indices

Atom index based definition of a centroid distance.

```
Indices
 ├ group1
 └ group2
```

| Name   | Type        | Default | Descriptions                               |
|--------|-------------|---------|--------------------------------------------|
| group1 | list\<int\> |         | Atom indices defining the first centroid.  |
| group2 | list\<int\> |         | Atom indices defining the second centroid. |

### Residues

Residue based definition of a centroid distance.

```
Residues
 ├ file_index1
 ├ residues1
 ├ file_index2
 └ residues2
```

| Name        | Type        | Default | Description                                                        |
|-------------|-------------|---------|--------------------------------------------------------------------|
| file_index1 | int         |         | Index of the file containing the residues for the first centroid.  |
| residues1   | list\<int\> |         | List of residues defining the first centroid.                      |
| file_index2 | int         |         | Index of the file containing the residues for the second centroid. |
| residues2   | list\<int\> |         | List of residues defining the second centroid.                     |

### Files

File based definition of a centroid distance.

```
Residues
 ├ file1
 └ file2
```

| Name  | Type | Default | Description                                                                |
|-------|------|---------|----------------------------------------------------------------------------|
| file1 | int  |         | Index of the file that contains the atoms that define the first centroid.  |
| file2 | int  |         | Index of the file that contains the atoms that define the second centroid. |

### RMSD

RMSD CV.

```
RMSD
 ├ reference_positions
 └ atom_indices
```

| Name                | Type          | Default | Description                                                                                                                              |
|---------------------|---------------|---------|------------------------------------------------------------------------------------------------------------------------------------------|
| reference_positions | list\<float\> |         | The reference positions from which to measure the RMSD, as a flat list of coordinates x1, y1, z1, x2, y2, z2, ...                        |
| atom_indices        | list\<int\>   |         | The atom indices used to measure the RMSD. The length of the reference positions list should be exactly 3 times the length of this list. |

### BiasPotential

Description of an external bias potential used in the simulation dynamics.

This type represents a discrimated union, and so only one of the fields may be present.

```
BiasPotential
 ├ BondLinear
 ├ RmsdExp
 ├ MovingBondHarmonic
 └ HarmonicCv
```

| Name               | Type                         | Default | Description                                               |
|--------------------|------------------------------|---------|-----------------------------------------------------------|
| BondLinear         | BondLinearParameters         |         | Linear bond length based force parameters.                |
| RmsdExp            | RmsdExpParameters            |         | Exponential RMSD based force parameters.                  |
| MovingBondHarmonic | MovingBondHarmonicParameters |         | Harmonic bond length force that moves at a constant speed |
| HarmonicCv         | HarmonicCVParameters         |         | Harmonic force that is defined on a given CV              |

### BondLinearParameters

Parameters for a linear force based on a bond distance.

```
BondLinearParameters
 ├ index1
 ├ index2
 ├ slope
 └ offset
```

| Name   | Type  | Default | Description                                                           |
|--------|-------|---------|-----------------------------------------------------------------------|
| index1 | int   |         | Atom index of the first atom defining the bond length.                |
| index2 | int   |         | Atom index of the first atom defining the bond length.                |
| slope  | float |         | Slope of the linear potential, i.e. `k` in `k * (distance - offset)`. |
| offset | float |         | Offset of the linear potential.                                       |

### RmsdExpParameters

Parameters for a force equal to the exponential of the RMSD between a subset of atoms and a reference set of positions.

```
RmsdExpParameters
 ├ reference_positions
 ├ atom_indices
 ├ height
 ├ sigma
 └ offset
```

| Name                | Type          | Default | Description                                                                                                                              |
|---------------------|---------------|---------|------------------------------------------------------------------------------------------------------------------------------------------|
| reference_positions | list\<float\> |         | The reference positions from which to measure the RMSD, as a flat list of coordinates x1, y1, z1, x2, y2, z2, ...                        |
| atom_indices        | list\<int\>   |         | The atom indices used to measure the RMSD. The length of the reference positions list should be exactly 3 times the length of this list. |
| height              | float         |         | Height parameter of the exponential function, i.e. `h` in the formula `h * exp(-0.5 * (rmsd - offset)^2 / sigma)`.                       |
| sigma               | float         |         | Sigma parameter of the exponential function.                                                                                             |
| offset              | float         |         | Offset parameter of the exponential function.                                                                                            |

### MovingHarmonicBondParameters

Parameters for a harmonic force based on a bond length that moves at a constant speed between to given lengths.

```
MovingHarmonicBondParameters
 ├ index1
 ├ index2
 ├ k
 ├ initial_offset_nm
 ├ final_offset_nm
 ├ resolution_nm
 └ rate_nm_per_ps
```

| Name              | Type  | Default | Description                                                                                                                                                 |
|-------------------|-------|---------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| index1            | int   |         | Atom index of the first atom defining the bond length.                                                                                                      |
| index2            | int   |         | Atom index of the first atom defining the bond length.                                                                                                      |
| k                 | float |         | Slope of the potential, i.e. `k` in `k * (distance - offset)^2`.                                                                                            |
| initial_offset_nm | float | null    | Initial offset of the potential. If not provided it will be set to the current bond length at the time when the bias potential becomes active.              |
| final_offset_nm   | float | null    | Final offset of the potential. If not provided it will be set to an estimate of the covalent bond length of the two atoms defined by `index1` and `index2`. |
| resolution_nm     | float | 0.01    | The resolution at which to update the offset.                                                                                                               |
| rate_nm_per_ps    | float |         | The speed at which the offset will "move".                                                                                                                  |

### HarmonicCVParameters

Parameters for a harmonic force that is defined by and acts upon a given collective variable.

```
HarmonicCVParameters
 ├ k
 ├ offset
 └ cv
```

| Name   | Type  | Default | Description                                                      |
|--------|-------|---------|------------------------------------------------------------------|
| k      | float |         | Slope of the potential, i.e. `k` in `k * (cv_value - offset)^2`. |
| offset | float |         | Offset for the potential.                                        |
| cv     | CV    |         | The collective variable that defines the force.                  |

### BiasSchedule

Schedule for when in the simulation to activate and deactivate the bias potentials defined in the parameters.

```
RmsdExpParameters
 ├ active_potentials
 └ triggers
```

| Name              | Type                    | Default | Description                                                                                                                                                                                                                                                                                                       |
|-------------------|-------------------------|---------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| active_potentials | list\<list\<int\>\>     |         | The length of this list is the number of separate phases in the simulation. Each list it contains is the indices of the bias potentials that are active during this phase (all other bias potentials will be turned off). Indices start at 0 and correspond to the order that they appear in the parameters file. |
| triggers          | list\<list\<Trigger\>\> |         | Each item of this list is a condition that once met will advance the bias schedule to the next phase. Therefore this should have length one less than the length of the `active_potentials` list. Each list it contains is a list of trigger conditions.                                                          |

### Trigger

Trigger bond condition between two atoms.

```
Trigger
 ├ index1
 ├ index2
 └ broken
```

| Name   | Type | Default | Description                                                               |
|--------|------|---------|---------------------------------------------------------------------------|
| index1 | int  |         | Atom index of the first atom defining the bond.                           |
| index2 | int  |         | Atom index of the second atom defining the bond.                          |
| broken | bool | false   | Whether the condition is defined by breaking instead of forming the bond. |

### TerminationCondition

Description of if and when to terminate the simulation early.

```
TerminationCondition
 ├ tail_steps
 └ bonds
```

| Name       | Type            | Default | Description                                                                             |
|------------|-----------------|---------|-----------------------------------------------------------------------------------------|
| tail_steps | int             | 100     | For how many steps to continue simulating after the termination condition has been met. |
| bonds      | list\<Trigger\> |         | The bonds defining the termination condition.                                           |

### Frames

Definition of which frames to save from the simulation and what information is contained in the frames.
Currently unused.

```
Frames
 ├ start
 ├ stop
 └ interval
```

| Name     | Type | Default | Description                                    |
|----------|------|---------|------------------------------------------------|
| start    | int  |         | First step to start recording.                 |
| stop     | int  |         | Last step to record.                           |
| interval | int  |         | The interval between steps at which to record. |

### Trajectory

Description of the trajectory, in xyz format, that will be recorded from the simulation.

```
Trajectory
 ├ interval
 ├ minimisation
 ├ forces
 ├ info
 ├ windowed
 ├ window_size
 └ residues
```

| Name         | Type                         | Default | Description                                                                                                       |
|--------------|------------------------------|---------|-------------------------------------------------------------------------------------------------------------------|
| interval     | int                          | 1       | Number of steps between recorded xyz frames.                                                                      |
| minimisation | bool                         | false   | Whether or not to include the the minimisation frames in the trajectory. Does nothing for classical minimisation. |
| forces       | string                       | "None"  | What forces to record in the xyz frames, if any. Options are "None", "Dynamics", "Bias", "Mtd".                   |
| info         | bool                         | false   | Whether to include the step number and system energy in the comment line in the xyz frames.                       |
| windowed     | bool                         | false   | Whether to only record windows around the trigger conditions in the bias schedule, if present.                    |
| window_size  | int                          | 200     | Size of windows that are recorded (centred on when the frame that the condition triggered).                       |
| residues     | list\<ResidueSpecification\> | []      | Which specific residues to record. If empty, the whole system is recorded.                                        |

### ResidueSpecification

Specification of a subset of residues from an input file.

```
ResidueSpecification
 ├ file_index
 └ residues
```

| Name       | Type        | Default | Description                                       |
|------------|-------------|---------|---------------------------------------------------|
| file_index | int         |         | Index of the input file that the residues are in. |
| residues   | list\<int\> |         | Indices of the residues in the input file.        |

### QDXFExport

Parameters for exporting information that can be used as inputs for PBSA calculations.

```
QDXFExport
 └ interval
```

| Name     | Type | Default | Description                                           |
|----------|------|---------|-------------------------------------------------------|
| interval | int  |         | Number of steps between frames that will be recorded. |

## Function Usage
```haskell
let
    options = {
        simulation = {
            step_size_ps = 0.001,
            stages = [
                {
                    Nvt = {
                        temperature_kelvin = 300,
                        steps = 1000
                    }
                }
            ]
        },
        trajectory = {
            interval = 100,
            info = true
        }
    },
    dojo = \protein_trc -> dojo_rex_s default_runspec options [protein_trc] []
in
    \protein_trc -> dojo protein_trc
```